In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
data_Dir = "../input"

# Any results you write to the current directory are saved as output.

['Watch_accelerometer.csv', 'Phones_accelerometer.csv', 'Phones_gyroscope.csv', 'readme.txt', 'Watch_gyroscope.csv']


In [2]:
df = pd.read_csv(data_Dir + "/Watch_accelerometer.csv" )
df2 = pd.read_csv(data_Dir + "/Watch_gyroscope.csv" )

In [3]:
# df = df.dropna(how='any',axis=0)
# df2 = df2.dropna(how='any',axis=0)

In [4]:
df[:20]

,Index,Arrival_Time,Creation_Time,x,y,z,User,Model,Device,gt
0,0,1424696638740,27920678471000,-0.565032,-9.572019,-0.614113,a,gear,gear_1,stand
1,1,1424696638740,27920681910000,-0.832584,-9.713276,-0.606930,a,gear,gear_1,stand
2,2,1424696638740,27920692014000,-1.018134,-9.935339,-0.544082,a,gear,gear_1,stand
3,3,1424696638741,27920701983000,-1.222838,-10.142437,-0.566229,a,gear,gear_1,stand
4,4,1424696638741,27920711906000,-1.577180,-10.480618,-0.402824,a,gear,gear_1,stand
5,5,1424696638741,27920721675000,-2.164358,-10.920552,-0.183755,a,gear,gear_1,stand
6,6,1424696638741,27920731721000,-2.973000,-11.063007,0.211887,a,gear,gear_1,stand
7,7,1424696638741,27920743061000,-3.888184,-11.082760,0.684742,a,gear,gear_1,stand
8,8,1424696638742,27920751586000,-4.891953,-10.890625,1.015740,a,gear,gear_1,stand
9,9,1424696638742,27920825873000,-12.600683,-7.674015,-1.179144,a,gear,gear_1,stand


In [5]:
df2.head()

,Index,Arrival_Time,Creation_Time,x,y,z,User,Model,Device,gt
0,0,1424696638743,27920678496000,-0.162187,-0.022104,0.059655,a,gear,gear_1,stand
1,1,1424696638743,27920681926000,-0.183225,-0.061785,0.012517,a,gear,gear_1,stand
2,2,1424696638743,27920692031000,-0.180829,-0.108657,-0.036485,a,gear,gear_1,stand
3,3,1424696638743,27920701997000,-0.147805,-0.157925,-0.098537,a,gear,gear_1,stand
4,7,1424696638744,27920743068000,0.182160,-0.323574,-0.277235,a,gear,gear_1,stand


In [6]:
len(df)

3540962

In [7]:
df = df.dropna(how='any',axis=0)

In [8]:
df = df.drop(labels = ['Arrival_Time','Creation_Time','Index', 'User'], axis=1)
df2 = df2.drop(labels = ['Arrival_Time','Creation_Time','Index', 'User','Model','Device'], axis=1)

In [9]:
length1 = len(df)
length2 = len(df2)
length = min(length1, length2)
df = df.head(length)
df2 = df2.head(length)
df2.columns = ['x1', 'y1', 'z1', 'gt1']
data = pd.concat([df, df2], axis=1)

In [10]:
to_drop = ['null']
data = data[~data['gt'].isin(to_drop)]
data = data[~data['gt1'].isin(to_drop)]

data = data.drop(labels = ['gt1'], axis=1)

data = data.iloc[::10, :]

cols_to_norm = ['x','y','z', 'x1','y1','z1']
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))



In [11]:
data.head()

,x,y,z,Model,Device,gt,x1,y1,z1
0,-0.000438,-0.180286,-0.068694,gear,gear_1,stand,-0.003297,-0.001565,0.003418
10,-0.220615,-0.052724,-0.047486,gear,gear_1,stand,0.013210,-0.032330,-0.055840
20,-0.221041,-0.033709,-0.072924,gear,gear_1,stand,-0.000254,0.000585,-0.003290
30,-0.224074,-0.027299,-0.072282,gear,gear_1,stand,-0.000173,-0.004702,-0.003021
40,-0.217735,-0.031882,-0.070206,gear,gear_1,stand,-0.000835,-0.002902,-0.002691


In [12]:
y = data[['gt']]
data = data.drop(labels = ['gt','Device','Model'], axis = 1)
data = pd.get_dummies(data)
parameters1 = len(data.columns)	

y = pd.get_dummies(y)
parameters2 = len(y.columns)


data = np.array(data)
y = np.array(y)

m = len(data)
crossval = round(3*m/4)
train_data = data[0:crossval,:]
train_data_y = y[0:crossval,:]

test_data = data[crossval:,:]
test_data_y = y[crossval:,:]

X_train = train_data
X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
X_train = np.array(X_train)

y_train = train_data_y

X_test = test_data
X_test = np.reshape(X_test, (X_test.shape[0],1,X_test.shape[1]))
X_test = np.array(X_test)

In [13]:
X_train[0]

array([[-0.00043755, -0.18028597, -0.06869415, -0.00329739, -0.00156469,
         0.00341772]])

In [14]:
y.shape

(344557, 6)

In [15]:
y[0]

array([0, 0, 0, 0, 1, 0], dtype=uint8)

In [16]:
y_test = test_data_y

In [17]:
Activity_dict = {'stand':1,'sit':2,'walk':3,'stairsup':4,'stairsdown':5,'bike':6}


In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD
from keras.preprocessing import  sequence

Using TensorFlow backend.


In [19]:
# model = Sequential()
# model.add(LSTM(24, input_dim = parameters1,return_sequences=True))
# model.add(LSTM(12,return_sequences=False))
# model.add(Dense(6, activation='softmax'))
# opt = SGD(lr=0.001, momentum=0.9, clipvalue=5.0)
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=3)

In [20]:
np.any(np.isnan(X_train))

True

In [21]:
np.all(np.isfinite(X_train))

False

In [22]:
np.isnan(X_train).sum()

103047

In [23]:
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

In [24]:
NeuralNetworkTrain(X_train,y_train,X_test,y_test)

NameError: name 'NeuralNetworkTrain' is not defined

In [25]:
model = Sequential()
model.add(LSTM(24, input_dim = parameters1,return_sequences=True))
model.add(LSTM(12))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=500,batch_size=1000)

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(24, return_sequences=True, input_shape=(None, 6))`
  


Instructions for updating:
Use tf.cast instead.
Train on 258418 samples, validate on 86139 samples
Epoch 1/500
258418/258418 [==============================] - 4s 17us/step - loss: 1.5198 - acc: 0.3424 - val_loss: 1.5842 - val_acc: 0.3247
Epoch 2/500
258418/258418 [==============================] - 2s 8us/step - loss: 1.3095 - acc: 0.3233 - val_loss: 1.7962 - val_acc: 0.2553
Epoch 3/500
258418/258418 [==============================] - 2s 8us/step - loss: 1.2641 - acc: 0.3316 - val_loss: 1.7965 - val_acc: 0.2623
Epoch 4/500
258418/258418 [==============================] - 2s 7us/step - loss: 1.2345 - acc: 0.3457 - val_loss: 1.7777 - val_acc: 0.2668
Epoch 5/500
258418/258418 [==============================] - 2s 8us/step - loss: 1.2024 - acc: 0.3587 - val_loss: 1.7894 - val_acc: 0.2844
Epoch 6/500
258418/258418 [==============================] - 2s 8us/step - loss: 1.1678 - acc: 0.3711 - val_loss: 1.8159 - val_acc: 0.3032
Epoch 7/500
258418/258418 [==============================] - 2s 8u